# RP3Beta runbook

In this notebook we run and save RP3Beta recommendations according to the best known model configuration.

## Loading libraries and setting parameters

In [1]:
import sys
from time import time

sys.path.append("../../..")

from src.common import config
from src.models.rp3beta import RP3Beta
from src.evaluation.evaluator import Evaluator

%load_ext autoreload

In [2]:
MODEL_NAME = "rp3beta"

paths = config.Paths(
    dataset_name="jobs_published-part",
    target_users_name="subset_validation",
    model_name=MODEL_NAME,
)

In [3]:
ALPHA = 1
BETA = 0

N_RECOMMENDATIONS = 10

## Model training

In [4]:
%%time

model = RP3Beta(alpha=ALPHA, beta=BETA)

model.load_interactions(paths.train)

model.preprocess()

model.fit()

CPU times: user 780 ms, sys: 40.4 ms, total: 820 ms
Wall time: 820 ms


## Generating recommendations

In [5]:
target_users = model.load_target_users(paths.target_users)

recommendations = model.recommend(
    target_users=target_users, n_recommendations=N_RECOMMENDATIONS
)

model.save_recommendations(recommendations, paths.recommendations)

17857it [00:05, 3147.99it/s]


## Evaluation

In [6]:
evaluator = Evaluator(
    recommendations_path=paths.recommendations_folder,
    test_path=paths.validation,
    k=10,
    models_to_evaluate=["rp3beta"],
)

evaluator.prepare()

evaluator.evaluate_models()

17857it [00:01, 16810.97it/s]


In [7]:
evaluator.evaluation_results

,precision,recall,F_1,F_05,ndcg,mAP,MRR,LAUC,HR,reco_in_test,test_coverage,Shannon,Gini,users_without_reco,users_without_k_reco
model_name,,,,,,,,,,,,,,,
rp3beta,0.025228,0.133893,0.039662,0.029355,0.087494,0.060082,0.095822,0.566309,0.213362,0.884695,0.847543,7.980025,0.690292,0.001792,0.005656
